In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
PROCESSED_DIR = '../data/processed/'
PROCESSED_FILENAME = "usdjpy-bar-2020-01-01-2024-12-31_processed.pkl"
PROCESSED_FILE_PATH = os.path.join(PROCESSED_DIR, PROCESSED_FILENAME)

In [3]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
df.head()

,timestamp,open,high,low,close,volume,time_group,close_delta,close_return,close_direction,label
1,2020-01-01 22:01:00,108.7570,108.7590,108.7495,108.7495,13300.000012,1,-0.0095,-0.000087,down,0
2,2020-01-01 22:02:00,108.7495,108.7535,108.7495,108.7535,4500.000000,1,0.0040,0.000037,up,2
3,2020-01-01 22:03:00,108.7540,108.7555,108.7535,108.7555,10490.000010,1,0.0020,0.000018,flat,1
4,2020-01-01 22:04:00,108.7575,108.7650,108.7555,108.7650,11600.000024,1,0.0095,0.000087,up,2
5,2020-01-01 22:05:00,108.7700,108.7700,108.7690,108.7700,1059.999987,1,0.0050,0.000046,up,2


In [5]:
df.describe()

,timestamp,open,high,low,close,volume,time_group,close_delta,close_return,label
count,1818676,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06,1.818676e+06
mean,2022-07-05 13:20:55.965527552,1.281896e+02,1.282000e+02,1.281792e+02,1.281896e+02,4.311489e+05,4.203974e+03,3.065142e-05,2.461495e-07,1.005389e+00
min,2020-01-01 22:01:00,1.012020e+02,1.013380e+02,1.011820e+02,1.012045e+02,0.000000e+00,1.000000e+00,-2.065000e+00,-1.380804e-02,0.000000e+00
25%,2021-04-01 18:33:45,1.094595e+02,1.094650e+02,1.094540e+02,1.094600e+02,1.180375e+05,1.847000e+03,-7.000000e-03,-5.705061e-05,0.000000e+00
50%,2022-07-11 20:03:30,1.312150e+02,1.312340e+02,1.311990e+02,1.312150e+02,2.726000e+05,4.956000e+03,0.000000e+00,0.000000e+00,1.000000e+00
75%,2023-10-04 07:22:15,1.456610e+02,1.456740e+02,1.456491e+02,1.456615e+02,5.858400e+05,6.143000e+03,7.500000e-03,5.846136e-05,2.000000e+00
max,2024-12-30 23:59:00,1.619500e+02,1.619510e+02,1.619435e+02,1.619495e+02,1.226455e+08,7.375000e+03,1.579500e+00,1.228753e-02,2.000000e+00
std,NaN,1.839441e+01,1.839714e+01,1.839161e+01,1.839442e+01,5.050698e+05,2.341641e+03,2.094446e-02,1.558500e-04,8.257596e-01


In [6]:
GROUP_COL = 'time_group'
SEQ_LEN = 30
HORIZON = 1

In [7]:
def get_sequence_start_indices(df, sequence_length=30, horizon=1, group_col='time_group'):
    indices = []

    for _, group in df.groupby(group_col):
        if len(group) >= sequence_length:
            group_indices = group.index.to_numpy()
            # Only take indices where a full sequence fits
            valid_starts = group_indices[:len(group) - sequence_length - horizon + 1]
            indices.extend(valid_starts)

    return indices


In [8]:
valid_start_idxs = get_sequence_start_indices(
    df, 
    sequence_length=30,
    horizon=1,
    group_col='time_group',
)

In [9]:
valid_start_idxs[0]

np.int64(1)

In [10]:
df.iloc[1:1+SEQ_LEN], df.iloc[1+SEQ_LEN: 1+SEQ_LEN+HORIZON]

(             timestamp      open      high       low     close         volume  \
 2  2020-01-01 22:02:00  108.7495  108.7535  108.7495  108.7535    4500.000000   
 3  2020-01-01 22:03:00  108.7540  108.7555  108.7535  108.7555   10490.000010   
 4  2020-01-01 22:04:00  108.7575  108.7650  108.7555  108.7650   11600.000024   
 5  2020-01-01 22:05:00  108.7700  108.7700  108.7690  108.7700    1059.999987   
 6  2020-01-01 22:06:00  108.7685  108.7685  108.7515  108.7570  603299.998939   
 7  2020-01-01 22:07:00  108.7595  108.7645  108.7510  108.7575  777979.996204   
 8  2020-01-01 22:08:00  108.7580  108.7635  108.7495  108.7545  251150.001049   
 9  2020-01-01 22:09:00  108.7565  108.7625  108.7435  108.7460  487400.000215   
 10 2020-01-01 22:10:00  108.7455  108.7480  108.7430  108.7450  145529.998422   
 11 2020-01-01 22:11:00  108.7445  108.7470  108.7395  108.7400   56379.999280   
 12 2020-01-01 22:12:00  108.7340  108.7600  108.7340  108.7575   95219.999909   
 13 2020-01-01 2

In [45]:
import torch
from torch.utils.data import Dataset

class ForexClassificationDataset(Dataset):
    """Dataset for sequence classification/forecasting with multi-step horizon."""

    def __init__(self, data, sequence_length, horizon, features, target, group_col='time_group'):
        self.sequence_length = sequence_length
        self.horizon = horizon
        self.features = features
        self.target = target
        self.group_col = group_col

        # Reset index to ensure integer indexing is valid
        self.data = data.reset_index(drop=True)
        self.feature_data = self.data[self.features].values
        self.target_data = self.data[self.target].values
        self.group_labels = self.data[self.group_col].values

        self.IDs = self._get_valid_sequence_starts()

    def _get_valid_sequence_starts(self):
        indices = []
        group_indices = {}

        for idx, group in enumerate(self.group_labels):
            group_indices.setdefault(group, []).append(idx)

        for group, idxs in group_indices.items():
            if len(idxs) >= self.sequence_length + self.horizon:
                valid = idxs[:len(idxs) - (self.sequence_length + self.horizon) + 1]
                indices.extend(valid)

        return indices

    def __len__(self):
        return len(self.IDs)

    def __getitem__(self, idx):
        start = self.IDs[idx]
        end = start + self.sequence_length
        target_idx = start + self.sequence_length + self.horizon - 1

        x = torch.tensor(self.feature_data[start:end], dtype=torch.float32)
        y = torch.tensor(self.target_data[target_idx], dtype=torch.long)

        return x, y


In [56]:
fx_dataset = ForexClassificationDataset(
    data=df, 
    sequence_length=SEQ_LEN, 
    horizon=HORIZON, 
    features=['close_return'],
    target='label',
    group_col='time_group'
)
    

In [60]:
fx_dataset[777][0].shape

torch.Size([30, 1])

## Good old train / test split

In [ ]:
from sklearn.model_selection import train_test_split

train_idx, val_idx = train_test_split(
    valid_start_idxs,
    test_size=0.2,
    shuffle=True,
    random_state=42
)

dataset = fx_dataset
train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)


In [ ]:
len(train_dataset), len(val_dataset)

## Stratified Shuffle

In [ ]:
valid_start_idxs[:5]

In [ ]:
target_indices = np.array(valid_start_idxs) + SEQ_LEN + HORIZON - 1


In [ ]:
target_indices

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

def get_stratified_split(df, valid_start_indices, sequence_length, horizon, target_col, test_size=0.2):
    valid_start_indices = np.array(valid_start_indices)  # ensure array

    target_indices = valid_start_indices + sequence_length + horizon - 1
    labels = df.loc[target_indices, target_col].values  # get labels efficiently

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
    train_idx, val_idx = next(splitter.split(valid_start_indices, labels))

    train_start_indices = valid_start_indices[train_idx]
    val_start_indices = valid_start_indices[val_idx]

    return train_start_indices.tolist(), val_start_indices.tolist()


In [ ]:
train_idx, test_idx = get_stratified_split(
    df,
    valid_start_indices=valid_start_idxs,
    sequence_length=SEQ_LEN,
    horizon=HORIZON,
    target_col='label'
)

In [ ]:
len(train_idx), len(test_idx)